# Operators
**This notebook demonstrates how to build and use operators in OpenFD. **

Operators are symbolic objects that act on expressions built out of grid functions (symbolic arrays).  

In [1]:
from openfd.alpha import Expression, GridFunction, Operator

In [2]:
u = GridFunction('u', shape=(10,))

In [3]:
v = GridFunction('v', shape=(10,))

## Identity operator
The default operator is the Identity operator

In [4]:
I = Operator('I')

The identity operator returns its operand unchanged.

In [5]:
expr = Expression(I(u))
expr

I(u)

In [6]:
expr[0]

u[0]

In [7]:
expr[1]

u[1]

In [8]:
expr = Expression(I*(u + v))

In [9]:
expr[0]

u[0] + v[0]

## Shift operator
Custom operators can be built by inheriting and overloading the Identity operator.
Let's build an operator that shifts the operand index + 1

In [10]:
class Shift(Operator):
    def __getitem__(self, index):
        return self.args[index+1]
        

In [11]:
S = Shift('S')

In [12]:
expr = Expression(S(u))

expr[0]

In [13]:
expr[1]

u[2]

## Matrix operators
There is a second way to define operators using the easy to read `*` notation. Any factors to the right of `*` are treated as the operand. This way of writing operators and operands is inspired by the way in which $A \mathbf{x}$ is used to denote matrix vector multiplication in linear algebra. Using this analogy, $A$ can be viewed as the operator and $\mathbf{x}$ its operand.

Let's begin by comparing the standard way of representing operators discussed above with the matrix notation way.
From the output we can tell in which mode the operators is acting in.

In [14]:
expr = Expression(S(u))
expr

S(u)

In [15]:
expr[0]

u[1]

In [16]:
expr = Expression(S*u)
expr

S*u

In [17]:
expr[0]

u[1]

## Operator nesting
One advantage with the matrix notation mode is that it can be used to nest operators to build complex expressions while still being very readable. 

In [18]:
expr = Expression(S(S(u)))
expr[0]

u[2]

In [19]:
expr = Expression(S*S*u)
expr[0]

u[2]

In [20]:
expr = Expression(u*S*S*(u + S*v))

In [21]:
expr[0]

u[0]*(u[2] + v[3])

However, some care is needed. It is not possible to isolate operands using the matrix mode. To demonstrate what we mean, let's first investigate the desired, correct, expression using the standard notation.


In [22]:
expr = Expression(S(S((u+v)))*u)
expr[0]

(u[2] + v[2])*u[0]

In [23]:
expr = Expression(S*S*(u+v)*v)
expr[0]

(u[2] + v[2])*v[2]

As we can see, the operator `S*` acts on all products to the right of it. A slightly more readible solution that gives the desired output is the following that combines both forms of writing operators.

In [24]:
expr = Expression(S(S*(u+v))*v)
expr[0]

(u[2] + v[2])*v[0]

## Operator expressions
Perhaps the greatest advantage with matrix operators is that the act of defining the operand can be postponed. Here is a simple example that defines a vector `expr` of operator expressions and another vector `vec` as its operand. We use standard lists to define the vectors and standard Python loops and sums to implement the dot product. 


In [28]:
expr = [u*S, v*S]
vec = [u, v]
result = sum([Expression(ei*vi) for ei, vi in zip(expr, vec)])
result

u*S*u + v*S*v